In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load
!pip install openpyxl
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import re
from unidecode import unidecode
from imblearn.under_sampling import NearMiss
import seaborn as sns
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
data_influenza = pd.read_csv("/kaggle/input/influenza/INFLUD21-10-01-2022.csv", on_bad_lines='skip', sep=';',low_memory=False)
data_influenza = data_influenza.fillna('n')
data_influenza_result = data_influenza[['PCR_RESUL','POS_PCRFLU','RES_AN',
                                        'TP_FLU_PCR','PCR_FLUASU','CLASSI_FIN',
                                        'PCR_SARS2','TP_FLU_AN','POS_AN_FLU',
                                        'AN_SARS2','VACINA_COV','DOSE_1_COV',
                                        'DOSE_2_COV','CS_SEXO','VACINA','FEBRE','TOSSE',
                                        'GARGANTA','DISPNEIA','DESC_RESP',
                                        'SATURACAO','DIARREIA','VOMITO','DOR_ABD',
                                        'FADIGA','PERD_OLFT','PERD_PALA','OUTRO_SIN',
                                        'OUTRO_DES']]
len(data_influenza_result)

# pre-processing infleunza dataset

In [ ]:
def print_value_counts(df):
    for c in df.columns:
        print(df[c].value_counts(),'\n')

def match_classes(dt):
    y=dt["Class"]
    x=dt.iloc[:,:13]
    x['index']=dt.index
    nm = NearMiss()
    x,y=nm.fit_resample(x,y)
    return x,y

def process_data(data):
    data = data[(data['FEBRE'] != 'n') & (data['FEBRE'] != 9.0) &
                (data['TOSSE'] != 'n') & (data['TOSSE'] != 9.0) &
                (data['GARGANTA'] != 'n') & (data['GARGANTA'] != 9.0) &
                (data['DISPNEIA'] != 'n') & (data['DISPNEIA'] != 9.0) &
                (data['DESC_RESP'] != 'n') & (data['DESC_RESP'] != 9.0) &
                (data['SATURACAO'] != 'n') & (data['SATURACAO'] != 9.0) &
                (data['DIARREIA'] != 'n') & (data['DIARREIA'] != 9.0) &
                (data['VOMITO'] != 'n') & (data['VOMITO'] != 9.0) &
                (data['DOR_ABD'] != 'n') & (data['DOR_ABD'] != 9.0) &
                (data['FADIGA'] != 'n') & (data['FADIGA'] != 9.0) &
                (data['PERD_OLFT'] != 'n') & (data['PERD_OLFT'] != 9.0) &
                (data['PERD_PALA'] != 'n') & (data['PERD_PALA'] != 9.0) &
                (data['CS_SEXO'] != 'n') & (data['CS_SEXO'] != 'I')]
    data['CS_SEXO'] = data['CS_SEXO'].replace('F', 1.0).replace('M', 2.0)

    return data

Base de dados com pacientes que não estão infectados por covid-19 e nem influenza(TESTE PCR+ANTIGENO)
1. Filtro de resultado do teste pcr negativo
2. Filtro de resultado do teste de antigeno negativo
3. Filtro pela classificação final igual SRAG não especificado

In [ ]:
df_not_covid_influ = data_influenza_result[(data_influenza_result['PCR_RESUL'] == 2.0) & 
                     (data_influenza_result['RES_AN'] == 2.0) &
                     (data_influenza_result['CLASSI_FIN'] == 4.0)]
df_not_covid_influ  = df_not_covid_influ[['CS_SEXO','FEBRE','TOSSE','GARGANTA','DISPNEIA',
                                         'DESC_RESP','SATURACAO','DIARREIA','VOMITO','DOR_ABD',
                                         'FADIGA','PERD_OLFT','PERD_PALA','OUTRO_SIN','OUTRO_DES']]
# df_not_covid_influ = process_data(df_not_covid_influ.copy())
df_not_covid_influ

Base de dados com pacientes que não estão infectados por covid-19 e nem influenza(TESTE PCR)
1. Filtro de resultado do teste pcr negativo
2. Filtro pela classificação final igual SRAG não especificado

In [ ]:
df_pcr_not_covid_influ = data_influenza_result[(data_influenza_result['PCR_RESUL'] == 2.0) & 
                     (data_influenza_result['CLASSI_FIN'] == 4.0)]
df_pcr_not_covid_influ  = df_pcr_not_covid_influ[['CS_SEXO','FEBRE','TOSSE','GARGANTA','DISPNEIA',
                                                  'DESC_RESP','SATURACAO','DIARREIA','VOMITO','DOR_ABD',
                                                  'FADIGA','PERD_OLFT','PERD_PALA','OUTRO_SIN','OUTRO_DES']]
# df_pcr_not_covid_influ = process_data(df_pcr_not_covid_influ.copy())
df_pcr_not_covid_influ

Base de dados com pacientes que não estão infectados por covid-19 e nem influenza(TESTE ANTIGENO)
1. Filtro de resultado do teste antigeno negativo
3. Filtro pela classificação final igual SRAG não especificado

In [ ]:
df_ant_not_covid_influ = data_influenza_result[(data_influenza_result['RES_AN'] == 2.0) &
                                           (data_influenza_result['CLASSI_FIN'] == 4.0)]
df_ant_not_covid_influ  = df_ant_not_covid_influ[['CS_SEXO','FEBRE','TOSSE','GARGANTA','DISPNEIA',
                                         'DESC_RESP','SATURACAO','DIARREIA','VOMITO','DOR_ABD',
                                         'FADIGA','PERD_OLFT','PERD_PALA','OUTRO_SIN','OUTRO_DES']]
# df_ant_not_covid_influ = process_data(df_ant_not_covid_influ.copy())
df_ant_not_covid_influ

Base de dados com pacientes que estão infectados pelo virus da influenza (TESTE PCR)
1. Filtro de resultado do teste pcr positivo
2. Filtro de resultado do teste pcr para influenza positivo
3. Filtro de resultado do teste antigeno para influenza negativo
4. Filtro de resultado do teste antigeno negativo
5. Filtro de resultado do teste pcr|ant para covid negativo
3. Filtro pela classificação final igual SRAG por influenza

In [ ]:
data_influenza_pcr = data_influenza_result[(data_influenza_result['CLASSI_FIN'] == 1.0) &
                                      (data_influenza_result['PCR_RESUL'] == 1.0) &
                                      (data_influenza_result['POS_PCRFLU'] == 1.0) &
                                      (data_influenza_result['RES_AN'] != 1.0) &
                                      (data_influenza_result['POS_AN_FLU'] != 1.0) &
                                      (data_influenza_result['AN_SARS2'] != 1.0)&
                                      (data_influenza_result['PCR_SARS2'] != 1.0)]
data_influenza_pcr = data_influenza_pcr[['TP_FLU_PCR','PCR_FLUASU','CS_SEXO','FEBRE','TOSSE','GARGANTA','DISPNEIA',
                                         'DESC_RESP','SATURACAO','DIARREIA','VOMITO','DOR_ABD',
                                         'FADIGA','PERD_OLFT','PERD_PALA','OUTRO_SIN','OUTRO_DES']]
# data_influenza_pcr = process_data(data_influenza_pcr.copy())
data_influenza_pcr
# print_value_counts(data_influenza_pcr)

Base de dados com pacientes que estão infectados pelo virus da influenza (TESTE ANTIGENO)
1. Filtro de resultado do teste antigeno positivo
2. Filtro de resultado do teste antigeno para influenza positivo
3. Filtro de resultado do teste pcr para influenza negativo
4. Filtro de resultado do teste pcr negativo
5. Filtro de resultado do teste pcr|ant para covid negativo
3. Filtro pela classificação final igual SRAG por influenza

In [ ]:
data_influenza_ant = data_influenza_result[(data_influenza_result['CLASSI_FIN'] == 1.0)&
                                      (data_influenza_result['RES_AN'] == 1.0) &
                                      (data_influenza_result['POS_AN_FLU'] == 1.0)&
                                      (data_influenza_result['PCR_RESUL'] != 1.0) &
                                      (data_influenza_result['POS_PCRFLU'] != 1.0)&
                                      (data_influenza_result['AN_SARS2'] != 1.0)&
                                      (data_influenza_result['PCR_SARS2'] != 1.0)]
data_influenza_ant = data_influenza_ant[['TP_FLU_AN','CS_SEXO','FEBRE','TOSSE','GARGANTA',
                                         'DISPNEIA','DESC_RESP','SATURACAO','DIARREIA',
                                         'VOMITO','DOR_ABD','FADIGA','PERD_OLFT','PERD_PALA',
                                         'OUTRO_SIN','OUTRO_DES']]
# data_influenza_ant = process_data(data_influenza_ant.copy())
# print_value_counts(data_influenza_ant)
data_influenza_ant

Base de dados com pacientes que estão infectados pelo virus da influenza (TESTE PCR + ANTIGENO)
1. Filtro de resultado do teste antigeno positivo
2. Filtro de resultado do teste antigeno para influenza positivo
3. Filtro de resultado do teste pcr para influenza positivo
4. Filtro de resultado do teste pcr positivo
5. Filtro de resultado do teste pcr|ant para covid negativo
3. Filtro pela classificação final igual SRAG por influenza

In [ ]:
data_influenza_ant_pcr = data_influenza_result[(data_influenza_result['CLASSI_FIN'] == 1.0)&
                                      (data_influenza_result['RES_AN'] == 1.0) &
                                      (data_influenza_result['POS_AN_FLU'] == 1.0)&
                                      (data_influenza_result['PCR_RESUL'] == 1.0) &
                                      (data_influenza_result['POS_PCRFLU'] == 1.0) &
                                      (data_influenza_result['AN_SARS2'] != 1.0)&
                                      (data_influenza_result['PCR_SARS2'] != 1.0)]
data_influenza_ant_pcr = data_influenza_ant_pcr[['TP_FLU_AN','TP_FLU_PCR','PCR_FLUASU',
                                                'CS_SEXO','FEBRE','TOSSE','GARGANTA',
                                                'DISPNEIA','DESC_RESP','SATURACAO','DIARREIA',
                                                'VOMITO','DOR_ABD','FADIGA','PERD_OLFT','PERD_PALA',
                                                'OUTRO_SIN','OUTRO_DES']]
# data_influenza_ant_pcr = process_data(data_influenza_ant_pcr.copy())
data_influenza_ant_pcr
# print_value_counts(data_influenza_ant_pcr)

Setting Final datasets (balanced and unbalanced)

database with both tests

In [ ]:
df_not_covid_influ['Class'] = 0 
data_influenza_pcr['Class'] = 1
data_influenza_ant['Class'] = 1
data_influenza_ant_pcr['Class'] = 1
df_both_test = pd.concat([df_not_covid_influ,data_influenza_pcr,data_influenza_ant,data_influenza_ant_pcr])
df_both_test = df_both_test[['CS_SEXO','FEBRE','TOSSE','GARGANTA','DISPNEIA','DESC_RESP','SATURACAO','DIARREIA',
                             'VOMITO','DOR_ABD','FADIGA','PERD_OLFT','PERD_PALA','Class']]
df_both_test_unb = df_both_test.copy()
# df_both ,y_both = match_classes(df_both_test)

# df_both['Class']=y_both
# del df_both['index']
df_both_test_unb ['Class'].value_counts()

In [ ]:
df_both_test_unb['Class'].value_counts()

database with antigen test only

In [ ]:
df_ant_not_covid_influ['Class'] = 0
data_influenza_ant['Class'] = 1
df_ant_test = pd.concat([df_ant_not_covid_influ,data_influenza_ant])
df_ant_test = df_ant_test[['CS_SEXO','FEBRE','TOSSE','GARGANTA','DISPNEIA','DESC_RESP','SATURACAO','DIARREIA',
                            'VOMITO','DOR_ABD','FADIGA','PERD_OLFT','PERD_PALA','Class']]
df_ant_test_unb = df_ant_test.copy()
df_ant,y_ant = match_classes(df_ant_test)

df_ant['Class']=y_ant
del df_ant['index']
df_ant['Class'].value_counts()

In [ ]:
df_ant_test_unb['Class'].value_counts()

database with pcr test only

In [ ]:
df_pcr_not_covid_influ['Class'] = 0
data_influenza_pcr['Class'] = 1
df_pcr_test = pd.concat([df_pcr_not_covid_influ,data_influenza_pcr])
df_pcr_test = df_pcr_test[['CS_SEXO','FEBRE','TOSSE','GARGANTA','DISPNEIA','DESC_RESP','SATURACAO','DIARREIA',
                            'VOMITO','DOR_ABD','FADIGA','PERD_OLFT','PERD_PALA','Class']]
df_pcr_test_unb = df_pcr_test.copy()
df_pcr,y_pcr=match_classes(df_pcr_test)

df_pcr['Class']=y_pcr
del df_pcr['index']
df_pcr['Class'].value_counts()

In [ ]:
df_pcr_test_unb['Class'].value_counts()

Resolving inconsistency

In [ ]:


def part_dataset(dt,sub):
    
    #part dataset with only class("class)") marked as negative
    neg=dt[dt["Class"]== 0]
    
    #part dataset with only class("class") marked as positive
    pos=dt[dt["Class"]==1]
    del neg["Class"]
    del pos["Class"]
    
    #part dataset with only unique values fom dataset pos
    pos_uniq=pos.drop_duplicates(subset=sub)
    
    neg_array=neg.iloc[:,:].values
    pos_uniq_array=pos_uniq.iloc[:,:].values
    pos_dupli_array=pos.iloc[:,:].values
    
    return neg,pos,neg_array, pos_uniq_array,pos_dupli_array

sub1=['CS_SEXO', 'FEBRE','TOSSE','GARGANTA','DISPNEIA','DESC_RESP',
      'SATURACAO','DIARREIA','VOMITO','DOR_ABD','FADIGA','PERD_OLFT',
      'PERD_PALA']

#PCR
neg_pcr,pos_pcr,neg_array_pcr,pos_uniq_array_pcr,pos_dupli_array_pcr=part_dataset(df_pcr.copy(),sub1)

#ANTIBODY
neg_ant,pos_ant,neg_array_ant,pos_uniq_array_ant,pos_dupli_array_ant=part_dataset(df_ant.copy(),sub1)  

#BOTH 
neg_both,pos_both,neg_array_both,pos_uniq_array_both,pos_dupli_array_both=part_dataset(df_both.copy(),sub1)



In [ ]:
def resolve_class_conflict(d_ori,pos_uniq_array,l_comp):
    lpp=dict()
    dict_count=dict()
    #run all the unique values from positive dataset
    for k in range(len(pos_uniq_array)):
        p=0
        lista_index=[]
        #run all the values to compare if is equal to the unique values from positive dataset
        for i in range(len(l_comp)):

            comparison=pos_uniq_array[k:k+1,:]==l_comp[i:i+1,:]
             #if the values are equal
            if(comparison.all()):
                #count how many equal values exist!
                p+=1
                #add l_comp value index in lista_index!
                lista_index.append(d_ori.iloc[i:i+1,:].index[0])
        #hold the number of same values
        dict_count[k]=p
        #hold all the lists of index
        lpp[k]=lista_index
    return dict_count,lpp
#pcr
dict_count_pcr,lpp_pcr=resolve_class_conflict(pos_pcr,pos_uniq_array_pcr,pos_dupli_array_pcr)
dict_count_neg_pcr,lpn_pcr=resolve_class_conflict(neg_pcr,pos_uniq_array_pcr, neg_array_pcr)

#antibody
dict_count_ant,lpp_ant=resolve_class_conflict(pos_ant,pos_uniq_array_ant,pos_dupli_array_ant)
dict_count_neg_ant,lpn_ant=resolve_class_conflict(neg_ant,pos_uniq_array_ant, neg_array_ant)

#both
#antibody
dict_count_both,lpp_both=resolve_class_conflict(pos_both,pos_uniq_array_both,pos_dupli_array_both)
dict_count_neg_both,lpn_both=resolve_class_conflict(neg_both,pos_uniq_array_both, neg_array_both)


In [ ]:
def get_conflict_values_keys(dict_count_neg,dict_count):
    dnp=pd.DataFrame(index=dict_count_neg.keys())
    dnp['neg']=dict_count_neg.values()
    dnp['pos']=dict_count.values()
    dnp=dnp[(dnp['neg']>0) & (dnp['pos']>0)]
    #get just the keys which the total rows number of neg values is less than pos values
    neg_smaller=dnp[dnp['neg'] < dnp['pos']]
    #get just the keys which the total rows number of pos values is less than neg values
    pos_smaller=dnp[dnp['neg'] > dnp['pos']]
    #get just the keys which the total rows number of pos values is equal to neg values
    #pos_neg_equal=dnp[dnp['neg'] == dnp['pos']]
    return neg_smaller, pos_smaller
#pcr
neg_smaller_pcr,pos_smaller_pcr=get_conflict_values_keys(dict_count_neg_pcr,dict_count_pcr)

#antibody
neg_smaller_ant,pos_smaller_ant=get_conflict_values_keys(dict_count_neg_ant,dict_count_ant)

#both
neg_smaller_both,pos_smaller_both=get_conflict_values_keys(dict_count_neg_both,dict_count_both)


In [ ]:
def get_index(data,l_data,l_index):
    
    for x in data.index:
        for k in l_data[x]:
            l_index.append(k)
    
       
    return l_index
#pcr
l_index_pcr=[]
l_index_pcr=get_index(neg_smaller_pcr,lpn_pcr,l_index_pcr)
l_index_pcr=get_index(pos_smaller_pcr,lpp_pcr,l_index_pcr)

#antibody
l_index_ant=[]
l_index_ant=get_index(neg_smaller_ant,lpn_ant,l_index_ant)
l_index_ant=get_index(pos_smaller_ant,lpp_ant,l_index_ant)

#both
l_index_both=[]
l_index_both=get_index(neg_smaller_both,lpn_both,l_index_both)
l_index_both=get_index(pos_smaller_both,lpp_both,l_index_both)

Drop inconsistency values!

In [ ]:
df_pcr=df_pcr.drop(l_index_pcr)
df_ant=df_ant.drop(l_index_ant)
df_both = df_both.drop(l_index_both)
df_pcr_unb = df_pcr.copy()
df_ant_unb = df_ant.copy()
df_both_unb = df_both.copy()

In [ ]:
df_both_test,y_both=match_classes(df_both )
df_both_test = df_both_test.replace(2,0)
df_both_test['Class']=y_both
del df_both_test['index']
df_both_test.to_csv('df_both_test.csv')
df_both_test.groupby('Class')['CS_SEXO', 'FEBRE','TOSSE','GARGANTA','DISPNEIA','DESC_RESP',
                    'SATURACAO','DIARREIA','VOMITO','DOR_ABD','FADIGA','PERD_OLFT',
                    'PERD_PALA'].agg(lambda x: x.value_counts().to_dict())

In [ ]:
df_both_test.info()

In [ ]:
df_pcr_test,y_pcr=match_classes(df_pcr)
df_pcr_test = df_pcr_test.replace(2,0)
df_pcr_test['Class']=y_pcr
del df_pcr_test['index']
df_pcr_test.to_csv('df_pcr_test.csv')
df_pcr_test.groupby('Class')['CS_SEXO', 'FEBRE','TOSSE','GARGANTA','DISPNEIA','DESC_RESP',
                    'SATURACAO','DIARREIA','VOMITO','DOR_ABD','FADIGA','PERD_OLFT',
                    'PERD_PALA'].agg(lambda x: x.value_counts().to_dict())

In [ ]:
df_pcr_test.info()

In [ ]:
df_ant_test,y_ant=match_classes(df_ant)
df_ant_test = df_ant_test.replace(2,0)
df_ant_test['Class']=y_ant
del df_ant_test['index']
df_ant_test.to_csv('df_ant_test.csv')
df_ant_test.groupby('Class')['CS_SEXO', 'FEBRE','TOSSE','GARGANTA','DISPNEIA','DESC_RESP',
                    'SATURACAO','DIARREIA','VOMITO','DOR_ABD','FADIGA','PERD_OLFT',
                    'PERD_PALA'].agg(lambda x: x.value_counts().to_dict())

In [ ]:
df_ant_test.info()

In [ ]:
df_both_unb.to_csv('df_both_unb.csv')
df_both_unb.groupby('Class')['CS_SEXO', 'FEBRE','TOSSE','GARGANTA','DISPNEIA','DESC_RESP',
                    'SATURACAO','DIARREIA','VOMITO','DOR_ABD','FADIGA','PERD_OLFT',
                    'PERD_PALA'].agg(lambda x: x.value_counts().to_dict())

In [ ]:
df_both_unb.info()

In [ ]:
df_pcr_unb.to_csv('df_pcr_unb.csv')
df_pcr_unb.groupby('Class')['CS_SEXO', 'FEBRE','TOSSE','GARGANTA','DISPNEIA','DESC_RESP',
                    'SATURACAO','DIARREIA','VOMITO','DOR_ABD','FADIGA','PERD_OLFT',
                    'PERD_PALA'].agg(lambda x: x.value_counts().to_dict())

In [ ]:
df_pcr_unb.info()

In [ ]:
df_ant_unb.to_csv('df_ant_unb.csv')
df_ant_unb.groupby('Class')['CS_SEXO', 'FEBRE','TOSSE','GARGANTA','DISPNEIA','DESC_RESP',
                    'SATURACAO','DIARREIA','VOMITO','DOR_ABD','FADIGA','PERD_OLFT',
                    'PERD_PALA'].agg(lambda x: x.value_counts().to_dict())

In [ ]:
df_ant_unb.info()

In [ ]:
# def update_symptoms_columns(data):
#     data_copy = data.copy()
#     Lsymt=data['OUTRO_DES']
#     headaches = ['CEFALEIA','DOR DE CABECA','CEFAL','CEFALEI','CEFALIA','CEF','CEFALE',
#                 'CAFALEIA','CEFA','DOR CABECA','CEFALEA','CEFAELIA','CEFALAEIA','CFALEIA',
#                 'CEFELEIA','CELAFEIA']
#     coryzas = ['CORIZA','CORIZ', 'CORISA','CORI']
#     k = 0
#     for symt in Lsymt:
#         #removing accentuation
#         symt=unidecode(symt)
#         symt=symt.upper()
#         #removing special symbols
#         symts=re.split( r'[),-;.+/(]',symt)
            
#         for s in symts:
#             split_symts = s.split()
#             for unique_symt in split_symts :
#                 unique_symt = unique_symt.strip()
                
#                 if unique_symt in coryzas:
#                     data_copy.iloc[k:k+1,13:14]=1.0
                    
#                 if unique_symt in headaches:
#                     data_copy.iloc[k:k+1,14:15]=1.0  
#         k+=1
        
#     return data_copy

# outro_sint = data_influenza_pcr_v2[data_influenza_pcr_v2['OUTRO_SIN'] == 1.0]
# del outro_sint ['OUTRO_SIN']
# outro_sint['CORIZA'] = 2.0
# outro_sint['DOR_CABECA'] = 2.0
# outro_sint.head(1)
# outro_sint_v2 = update_symptoms_columns(outro_sint)